# 📊 02 - Carregamento e Processamento de Dados

## 📖 Visão Geral

Este notebook é responsável pelo carregamento, consolidação e preprocessamento dos dados do Web of Science em formato Excel.

### 🎯 Responsabilidades

- ✅ Carregamento de 39 arquivos Excel do Web of Science
- ✅ Consolidação em dataset único
- ✅ Mapeamento e padronização de colunas
- ✅ Validação e limpeza básica dos dados
- ✅ Análise exploratória inicial
- ✅ Geração de estatísticas de qualidade

### 📦 Dependências

- pandas
- xlrd (para arquivos .xls)
- numpy
- pathlib

### 🔗 Notebooks Relacionados

- **Anterior**: `01_configuracao_sistema.ipynb`
- **Próximo**: `03_analise_regex.ipynb`

In [1]:
# 📦 Imports e carregamento de configurações

import pandas as pd
import numpy as np
import os
import glob
import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Tuple

# Carregar configurações do sistema
config_file = 'config_sistema.json'
if os.path.exists(config_file):
    with open(config_file, 'r', encoding='utf-8') as f:
        config_data = json.load(f)
    
    SISTEMA_CONFIGURADO = config_data.get('sistema_configurado', False)
    PATHS = config_data.get('paths', {})
    CONFIG = config_data.get('config', {})
    
    print("✅ Configurações carregadas do notebook anterior")
    print(f"   Sistema configurado: {SISTEMA_CONFIGURADO}")
else:
    print("❌ Arquivo de configuração não encontrado!")
    print("💡 Execute primeiro o notebook '01_configuracao_sistema.ipynb'")
    raise FileNotFoundError("Configuração do sistema necessária")

# Verificar se há arquivos Excel
excel_dir = PATHS.get('excel_files', '/home/delon/Modelos/modeloCenanoInk/data/raw/arquivos_excel_artigo_cienciometrico/')
print(f"📁 Diretório de dados: {excel_dir}")

✅ Configurações carregadas do notebook anterior
   Sistema configurado: True
📁 Diretório de dados: /home/delon/Modelos/modeloCenanoInk/data/raw/arquivos_excel_artigo_cienciometrico/


In [2]:
# 🔍 Exploração inicial dos arquivos Excel

def explorar_arquivos_excel(diretorio: str) -> Dict[str, any]:
    """Explora os arquivos Excel disponíveis e retorna estatísticas"""
    
    print("🔍 EXPLORANDO ARQUIVOS EXCEL")
    print("=" * 35)
    
    # Buscar arquivos Excel
    excel_files = glob.glob(os.path.join(diretorio, '*.xls'))
    
    if not excel_files:
        print(f"❌ Nenhum arquivo Excel encontrado em: {diretorio}")
        return None
    
    print(f"📊 Total de arquivos encontrados: {len(excel_files)}")
    
    # Analisar alguns arquivos para entender estrutura
    exploracao = {
        'total_arquivos': len(excel_files),
        'arquivos': [],
        'tamanhos': [],
        'estruturas': [],
        'total_estimado_registros': 0
    }
    
    print("\n📋 Analisando estrutura dos primeiros 3 arquivos:")
    
    for i, arquivo_path in enumerate(sorted(excel_files)[:3]):
        nome_arquivo = os.path.basename(arquivo_path)
        tamanho_mb = os.path.getsize(arquivo_path) / (1024 * 1024)
        
        print(f"\n📄 Arquivo {i+1}: {nome_arquivo} ({tamanho_mb:.1f} MB)")
        
        try:
            # Tentar ler o arquivo
            df_temp = pd.read_excel(arquivo_path, engine='xlrd')
            
            estrutura = {
                'arquivo': nome_arquivo,
                'linhas': len(df_temp),
                'colunas': len(df_temp.columns),
                'tamanho_mb': tamanho_mb,
                'colunas_principais': list(df_temp.columns[:10])
            }
            
            exploracao['arquivos'].append(nome_arquivo)
            exploracao['tamanhos'].append(tamanho_mb)
            exploracao['estruturas'].append(estrutura)
            
            print(f"   📊 Dimensões: {len(df_temp):,} linhas × {len(df_temp.columns)} colunas")
            print(f"   📋 Primeiras colunas: {', '.join(df_temp.columns[:5])}...")
            
            # Verificar colunas essenciais do Web of Science
            colunas_wos = ['Article Title', 'Abstract', 'Authors', 'Publication Year']
            colunas_encontradas = [col for col in colunas_wos if col in df_temp.columns]
            print(f"   ✅ Colunas WoS encontradas: {len(colunas_encontradas)}/{len(colunas_wos)}")
            
        except Exception as e:
            print(f"   ❌ Erro ao ler arquivo: {e}")
            continue
    
    # Estimativa total
    if exploracao['estruturas']:
        media_linhas = np.mean([est['linhas'] for est in exploracao['estruturas']])
        exploracao['total_estimado_registros'] = int(media_linhas * len(excel_files))
        
        print(f"\n📊 Estimativas totais:")
        print(f"   📈 Registros por arquivo: ~{media_linhas:,.0f}")
        print(f"   📈 Total estimado de registros: ~{exploracao['total_estimado_registros']:,}")
        print(f"   💾 Tamanho total: ~{sum(exploracao['tamanhos']):.1f} MB")
    
    return exploracao

# Executar exploração
exploracao_inicial = explorar_arquivos_excel(excel_dir)

🔍 EXPLORANDO ARQUIVOS EXCEL
📊 Total de arquivos encontrados: 39

📋 Analisando estrutura dos primeiros 3 arquivos:

📄 Arquivo 1: savedrecs1.xls (4.3 MB)
   📊 Dimensões: 1,000 linhas × 72 colunas
   📋 Primeiras colunas: Publication Type, Authors, Book Authors, Book Editors, Book Group Authors...
   ✅ Colunas WoS encontradas: 4/4

📄 Arquivo 2: savedrecs10.xls (4.3 MB)
   📊 Dimensões: 1,000 linhas × 72 colunas
   📋 Primeiras colunas: Publication Type, Authors, Book Authors, Book Editors, Book Group Authors...
   ✅ Colunas WoS encontradas: 4/4

📄 Arquivo 3: savedrecs11.xls (4.3 MB)
   📊 Dimensões: 1,000 linhas × 72 colunas
   📋 Primeiras colunas: Publication Type, Authors, Book Authors, Book Editors, Book Group Authors...
   ✅ Colunas WoS encontradas: 4/4

📊 Estimativas totais:
   📈 Registros por arquivo: ~1,000
   📈 Total estimado de registros: ~39,000
   💾 Tamanho total: ~12.8 MB
   📊 Dimensões: 1,000 linhas × 72 colunas
   📋 Primeiras colunas: Publication Type, Authors, Book Authors, Boo

In [3]:
# 📥 Função de carregamento consolidado

def carregar_arquivos_excel_wos(diretorio_excel: str = None) -> pd.DataFrame:
    """Carrega todos os arquivos Excel do Web of Science e consolida em um DataFrame único"""
    
    if diretorio_excel is None:
        diretorio_excel = excel_dir
    
    print("📊 CARREGANDO ARQUIVOS EXCEL WOS")
    print("=" * 40)
    
    # Buscar todos os arquivos Excel
    excel_files = glob.glob(os.path.join(diretorio_excel, '*.xls'))
    
    if not excel_files:
        print("❌ Nenhum arquivo Excel encontrado!")
        return None
    
    print(f"📁 Encontrados {len(excel_files)} arquivos Excel")
    
    dataframes = []
    total_records = 0
    errors = []
    
    for i, file_path in enumerate(excel_files, 1):
        try:
            print(f"\n📄 Carregando arquivo {i}/{len(excel_files)}: {os.path.basename(file_path)}")
            
            # Carregar arquivo Excel
            df_temp = pd.read_excel(file_path, engine='xlrd')
            
            # Adicionar informações de origem
            df_temp['arquivo_origem'] = os.path.basename(file_path)
            df_temp['batch_numero'] = i
            df_temp['data_carregamento'] = datetime.now().isoformat()
            
            print(f"  ✅ Carregado: {len(df_temp):,} registros")
            
            dataframes.append(df_temp)
            total_records += len(df_temp)
            
        except Exception as e:
            error_msg = f"Erro no arquivo {os.path.basename(file_path)}: {str(e)}"
            print(f"  ❌ {error_msg}")
            errors.append(error_msg)
    
    if not dataframes:
        print("❌ Nenhum arquivo foi carregado com sucesso!")
        return None
    
    # Consolidar todos os DataFrames
    print(f"\n🔄 Consolidando {len(dataframes)} DataFrames...")
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    
    print(f"\n✅ CONSOLIDAÇÃO CONCLUÍDA")
    print(f"📊 Total de registros: {len(df_consolidado):,}")
    print(f"📋 Total de colunas: {len(df_consolidado.columns)}")
    print(f"📁 Arquivos processados: {len(dataframes)}/{len(excel_files)}")
    
    if errors:
        print(f"\n⚠️ Erros encontrados ({len(errors)}):")
        for error in errors[:3]:  # Mostrar apenas os primeiros 3 erros
            print(f"  - {error}")
        if len(errors) > 3:
            print(f"  ... e mais {len(errors) - 3} erros")
    
    return df_consolidado

# Executar carregamento
print("🚀 Iniciando carregamento dos dados...")
df_raw = carregar_arquivos_excel_wos()

if df_raw is not None:
    print(f"\n🎉 Dados carregados com sucesso!")
    print(f"📊 Dataset shape: {df_raw.shape}")
else:
    print("❌ Falha no carregamento dos dados")

🚀 Iniciando carregamento dos dados...
📊 CARREGANDO ARQUIVOS EXCEL WOS
📁 Encontrados 39 arquivos Excel

📄 Carregando arquivo 1/39: savedrecs26.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 2/39: savedrecs20.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 2/39: savedrecs20.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 3/39: savedrecs3.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 3/39: savedrecs3.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 4/39: savedrecs6.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 4/39: savedrecs6.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 5/39: savedrecs22.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 5/39: savedrecs22.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 6/39: savedrecs18.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 6/39: savedrecs18.xls
  ✅ Carregado: 1,000 registros

📄 Carregando arquivo 7/39: savedrecs5.xls
  ✅ Carregado: 1,000 registros

📄 

In [4]:
# 🔍 Filtro de registros por acesso aberto (Open Access)

def filtrar_registros_open_access(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra os registros que possuem classificação de Open Access
    
    Args:
        df (pd.DataFrame): DataFrame com dados processados
        
    Returns:
        pd.DataFrame: DataFrame contendo apenas registros com classificação de Open Access
    """
    print("\n🔒 FILTRANDO REGISTROS POR OPEN ACCESS")
    print("=" * 40)
    
    df_filtrado = df.copy()
    
    # Verificar se a coluna Open Access está disponível
    coluna_oa = None
    possiveis_colunas = ['Open Access Designations', 'Open Access', 'OA', 'Acesso Aberto']
    
    for col in possiveis_colunas:
        if col in df_filtrado.columns:
            coluna_oa = col
            break
    
    if coluna_oa is None:
        print("⚠️ Nenhuma coluna de Open Access encontrada, utilizando dados sem filtro")
        # Adicionar flag mesmo assim para manter consistência
        df_filtrado['filtrado_open_access'] = False
        return df_filtrado
    
    # Filtrar registros com classificação de Open Access (valores não vazios)
    registros_iniciais = len(df_filtrado)
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].notna()]
    
    # Excluir registros com valores vazios ou placeholders
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].astype(str).str.strip() != '']
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].astype(str).str.lower() != 'nan']
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].astype(str).str.lower() != 'none']
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].astype(str).str.lower() != 'unknown']
    
    registros_finais = len(df_filtrado)
    registros_removidos = registros_iniciais - registros_finais
    
    # Analisar distribuição das categorias de Open Access
    if registros_finais > 0:
        categorias_oa = df_filtrado[coluna_oa].value_counts()
        
        print(f"\n📊 Distribuição das categorias de Open Access:")
        for categoria, contagem in categorias_oa.items():
            percentual = (contagem / registros_finais) * 100
            print(f"   {categoria}: {contagem:,} registros ({percentual:.1f}%)")
    
    print(f"\n📊 Resumo do filtro de Open Access:")
    print(f"   📥 Registros iniciais: {registros_iniciais:,}")
    print(f"   📤 Registros finais: {registros_finais:,}")
    print(f"   🗑️ Registros removidos (sem OA): {registros_removidos:,} ({(registros_removidos/registros_iniciais)*100:.1f}%)")
    
    # Criar flag para indicar filtro aplicado
    df_filtrado['filtrado_open_access'] = True
    
    return df_filtrado

# Aplicar filtro de Open Access após limpeza básica
if 'df_clean' in locals() and df_clean is not None:
    df_clean = filtrar_registros_open_access(df_clean)
    print("🔒 Filtro de Open Access aplicado e integrado ao fluxo principal")
else:
    print("⚠️ Pulando filtro de Open Access - dados não disponíveis")

⚠️ Pulando filtro de Open Access - dados não disponíveis


In [5]:
# 🗺️ Mapeamento e padronização de colunas

def mapear_colunas_wos(df: pd.DataFrame) -> pd.DataFrame:
    """Mapeia colunas do formato Web of Science para formato padrão do sistema"""
    
    print("\n🗺️ MAPEANDO COLUNAS WOS PARA FORMATO PADRÃO")
    print("=" * 45)
    
    # Mapeamento de colunas WoS para formato padrão
    mapeamento_colunas = {
        'Article Title': 'Article Title',
        'Abstract': 'Abstract',
        'Author Keywords': 'Author Keywords',
        'Keywords Plus': 'Keywords Plus',
        'Authors': 'Authors',
        'Publication Year': 'Publication Year',
        'Source Title': 'Source Title',
        'DOI': 'DOI',
        'Addresses': 'Addresses',
        'Research Areas': 'Research Areas',
        'Web of Science Categories': 'WoS Categories',
        'Times Cited': 'Times Cited',
        'Journal': 'Journal',
        'Volume': 'Volume',
        'Issue': 'Issue',
        'Pages': 'Pages'
    }
    
    # Identificar colunas disponíveis
    colunas_disponiveis = list(df.columns)
    colunas_mapeadas = {}
    colunas_nao_encontradas = []
    
    print("📋 Verificando mapeamento de colunas:")
    
    for coluna_wos, coluna_padrao in mapeamento_colunas.items():
        if coluna_wos in colunas_disponiveis:
            colunas_mapeadas[coluna_wos] = coluna_padrao
            print(f"  ✅ {coluna_wos} → {coluna_padrao}")
        else:
            colunas_nao_encontradas.append(coluna_wos)
            print(f"  ⚠️ {coluna_wos} → NÃO ENCONTRADA")
    
    # Criar DataFrame mapeado
    df_mapeado = df.copy()
    
    # Renomear colunas encontradas
    if colunas_mapeadas:
        df_mapeado = df_mapeado.rename(columns=colunas_mapeadas)
        print(f"\n✅ {len(colunas_mapeadas)} colunas mapeadas com sucesso")
    
    # Garantir colunas essenciais
    colunas_essenciais = ['Article Title', 'Abstract', 'Authors', 'Publication Year']
    colunas_faltantes = []
    
    for coluna in colunas_essenciais:
        if coluna not in df_mapeado.columns:
            colunas_faltantes.append(coluna)
    
    if colunas_faltantes:
        print(f"\n⚠️ Colunas essenciais não encontradas: {colunas_faltantes}")
        print("💡 O sistema tentará continuar, mas a análise pode ser limitada")
    else:
        print(f"\n✅ Todas as colunas essenciais estão disponíveis")
    
    print(f"\n📊 DataFrame final: {len(df_mapeado):,} linhas x {len(df_mapeado.columns)} colunas")
    
    return df_mapeado

# Aplicar mapeamento se dados foram carregados
if df_raw is not None:
    df_mapped = mapear_colunas_wos(df_raw)
    print("🗺️ Mapeamento de colunas concluído")
else:
    print("⚠️ Pulando mapeamento - dados não carregados")


🗺️ MAPEANDO COLUNAS WOS PARA FORMATO PADRÃO
📋 Verificando mapeamento de colunas:
  ✅ Article Title → Article Title
  ✅ Abstract → Abstract
  ✅ Author Keywords → Author Keywords
  ✅ Keywords Plus → Keywords Plus
  ✅ Authors → Authors
  ✅ Publication Year → Publication Year
  ✅ Source Title → Source Title
  ✅ DOI → DOI
  ✅ Addresses → Addresses
  ✅ Research Areas → Research Areas
  ⚠️ Web of Science Categories → NÃO ENCONTRADA
  ⚠️ Times Cited → NÃO ENCONTRADA
  ⚠️ Journal → NÃO ENCONTRADA
  ✅ Volume → Volume
  ✅ Issue → Issue
  ⚠️ Pages → NÃO ENCONTRADA



✅ 12 colunas mapeadas com sucesso

✅ Todas as colunas essenciais estão disponíveis

📊 DataFrame final: 38,323 linhas x 75 colunas
🗺️ Mapeamento de colunas concluído


In [6]:
# 🔍 Validação, Limpeza e Processamento dos Dados

def validar_dados_wos(df: pd.DataFrame) -> Dict[str, any]:
    """Valida a qualidade dos dados WoS carregados"""
    
    print("\n🔍 VALIDANDO QUALIDADE DOS DADOS WOS")
    print("=" * 40)
    
    validacao = {
        'total_registros': len(df),
        'colunas_disponiveis': list(df.columns),
        'abstracts_validos': 0,
        'titulos_validos': 0,
        'anos_validos': 0,
        'autores_validos': 0,
        'duplicatas_potenciais': 0,
        'registros_completos': 0,
        'qualidade_geral': 0,
        'nano_relevancia_titulo': 0,
        'nano_relevancia_abstract': 0
    }
    
    # Validar Abstract
    if 'Abstract' in df.columns:
        abstracts_nao_nulos = df['Abstract'].notna()
        abstracts_nao_vazios = df['Abstract'].astype(str).str.strip().str.len() > 50
        validacao['abstracts_validos'] = (abstracts_nao_nulos & abstracts_nao_vazios).sum()
        print(f"📄 Abstracts válidos: {validacao['abstracts_validos']:,}/{len(df):,} ({validacao['abstracts_validos']/len(df)*100:.1f}%)")
    
    # Validar Article Title
    if 'Article Title' in df.columns:
        titulos_nao_nulos = df['Article Title'].notna()
        titulos_nao_vazios = df['Article Title'].astype(str).str.strip().str.len() > 10
        validacao['titulos_validos'] = (titulos_nao_nulos & titulos_nao_vazios).sum()
        print(f"📰 Títulos válidos: {validacao['titulos_validos']:,}/{len(df):,} ({validacao['titulos_validos']/len(df)*100:.1f}%)")
    
    # Validar Publication Year
    if 'Publication Year' in df.columns:
        try:
            anos_numericos = pd.to_numeric(df['Publication Year'], errors='coerce')
            anos_validos_mask = (anos_numericos >= 1990) & (anos_numericos <= 2024)
            validacao['anos_validos'] = anos_validos_mask.sum()
            print(f"📅 Anos válidos: {validacao['anos_validos']:,}/{len(df):,} ({validacao['anos_validos']/len(df)*100:.1f}%)")
        except:
            print(f"⚠️ Erro na validação de anos")
    
    # Validar Authors
    if 'Authors' in df.columns:
        autores_nao_nulos = df['Authors'].notna()
        autores_nao_vazios = df['Authors'].astype(str).str.strip().str.len() > 3
        validacao['autores_validos'] = (autores_nao_nulos & autores_nao_vazios).sum()
        print(f"👥 Autores válidos: {validacao['autores_validos']:,}/{len(df):,} ({validacao['autores_validos']/len(df)*100:.1f}%)")
    
    # Verificar duplicatas potenciais por título
    if 'Article Title' in df.columns:
        duplicatas = df['Article Title'].duplicated().sum()
        validacao['duplicatas_potenciais'] = duplicatas
        print(f"🔄 Duplicatas potenciais: {duplicatas:,} ({duplicatas/len(df)*100:.1f}%)")
    
    # Calcular registros completos
    colunas_essenciais = ['Article Title', 'Abstract', 'Authors', 'Publication Year']
    colunas_presentes = [col for col in colunas_essenciais if col in df.columns]
    
    if colunas_presentes:
        mask_completos = df[colunas_presentes].notna().all(axis=1)
        validacao['registros_completos'] = mask_completos.sum()
        print(f"📋 Registros completos: {validacao['registros_completos']:,}/{len(df):,} ({validacao['registros_completos']/len(df)*100:.1f}%)")
    
    # Calcular qualidade geral
    scores = []
    if validacao['abstracts_validos'] > 0:
        scores.append(validacao['abstracts_validos'] / len(df) * 100)
    if validacao['titulos_validos'] > 0:
        scores.append(validacao['titulos_validos'] / len(df) * 100)
    if validacao['anos_validos'] > 0:
        scores.append(validacao['anos_validos'] / len(df) * 100)
    if validacao['autores_validos'] > 0:
        scores.append(validacao['autores_validos'] / len(df) * 100)
    
    if scores:
        validacao['qualidade_geral'] = sum(scores) / len(scores)
        print(f"\n📊 Qualidade geral dos dados: {validacao['qualidade_geral']:.1f}/100")
    
    # Análise de nano-relevância
    print(f"\n🔬 ANÁLISE DE RELEVÂNCIA PARA NANOTECNOLOGIA:")
    nano_keywords = ['nano', 'nanoparticle', 'nanotechnology', 'nanomaterial', 'coating', 'paint', 'revestimento']
    
    if 'Article Title' in df.columns:
        nano_mentions_title = 0
        for keyword in nano_keywords:
            count = df['Article Title'].astype(str).str.lower().str.contains(keyword, na=False).sum()
            if count > 0:
                print(f"  📰 '{keyword}' em títulos: {count} menções")
                nano_mentions_title += count
        validacao['nano_relevancia_titulo'] = nano_mentions_title
    
    if 'Abstract' in df.columns:
        nano_mentions_abstract = 0
        for keyword in nano_keywords:
            count = df['Abstract'].astype(str).str.lower().str.contains(keyword, na=False).sum()
            if count > 0:
                print(f"  📄 '{keyword}' em abstracts: {count} menções")
                nano_mentions_abstract += count
        validacao['nano_relevancia_abstract'] = nano_mentions_abstract
    
    return validacao

def limpar_dados_basico(df: pd.DataFrame) -> pd.DataFrame:
    """Aplica limpeza básica nos dados carregados"""
    
    print("\n🧹 APLICANDO LIMPEZA BÁSICA DOS DADOS")
    print("=" * 40)
    
    df_clean = df.copy()
    linhas_iniciais = len(df_clean)
    
    # 1. Remover duplicatas exatas
    duplicatas_exatas = df_clean.duplicated().sum()
    if duplicatas_exatas > 0:
        print(f"🔄 Removendo {duplicatas_exatas:,} duplicatas exatas")
        df_clean = df_clean.drop_duplicates()
    else:
        print("✅ Nenhuma duplicata exata encontrada")
    
    # 2. Limpeza de abstracts
    if 'Abstract' in df_clean.columns:
        print(f"📄 Limpando abstracts...")
        df_clean['Abstract'] = df_clean['Abstract'].astype(str)
        df_clean['Abstract'] = df_clean['Abstract'].str.replace('\n', ' ').str.replace('\r', ' ')
        df_clean['Abstract'] = df_clean['Abstract'].str.replace('\t', ' ').str.strip()
        df_clean['Abstract'] = df_clean['Abstract'].str.replace(r'\s+', ' ', regex=True)
    
    # 3. Limpeza de títulos
    if 'Article Title' in df_clean.columns:
        print(f"📰 Limpando títulos...")
        df_clean['Article Title'] = df_clean['Article Title'].astype(str).str.strip()
    
    # 4. Limpeza de anos
    if 'Publication Year' in df_clean.columns:
        print(f"📅 Validando anos de publicação...")
        df_clean['Publication Year'] = pd.to_numeric(df_clean['Publication Year'], errors='coerce')
    
    linhas_finais = len(df_clean)
    print(f"\n📊 Resumo da limpeza:")
    print(f"   📥 Registros iniciais: {linhas_iniciais:,}")
    print(f"   📤 Registros finais: {linhas_finais:,}")
    
    return df_clean

def filtrar_registros_open_access(df: pd.DataFrame) -> pd.DataFrame:
    """Filtra os registros que possuem classificação de Open Access"""
    
    print("\n🔒 FILTRANDO REGISTROS POR OPEN ACCESS")
    print("=" * 40)
    
    df_filtrado = df.copy()
    coluna_oa = None
    possiveis_colunas = ['Open Access Designations', 'Open Access', 'OA', 'Acesso Aberto']
    
    for col in possiveis_colunas:
        if col in df_filtrado.columns:
            coluna_oa = col
            break
    
    if coluna_oa is None:
        print("⚠️ Nenhuma coluna de Open Access encontrada, utilizando dados sem filtro")
        df_filtrado['filtrado_open_access'] = False
        return df_filtrado
    
    registros_iniciais = len(df_filtrado)
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].notna()]
    df_filtrado = df_filtrado[df_filtrado[coluna_oa].astype(str).str.strip() != '']
    df_filtrado = df_filtrado[~df_filtrado[coluna_oa].astype(str).str.lower().isin(['nan', 'none', 'unknown'])]
    
    registros_finais = len(df_filtrado)
    registros_removidos = registros_iniciais - registros_finais
    
    print(f"\n📊 Resumo do filtro de Open Access:")
    print(f"   📥 Registros iniciais: {registros_iniciais:,}")
    print(f"   📤 Registros finais: {registros_finais:,}")
    print(f"   🗑️ Registros removidos: {registros_removidos:,}")
    
    df_filtrado['filtrado_open_access'] = True
    return df_filtrado

# Executar pipeline de processamento
if 'df_mapped' in locals() and df_mapped is not None:
    print("\n🔄 INICIANDO PIPELINE DE PROCESSAMENTO")
    print("=" * 40)
    
    # 1. Validação
    print("\n📋 Etapa 1: Validação dos dados")
    validacao_resultado = validar_dados_wos(df_mapped)
    
    # 2. Limpeza básica
    print("\n🧹 Etapa 2: Limpeza básica")
    df_clean = limpar_dados_basico(df_mapped)
    
    # 3. Filtro Open Access
    print("\n🔒 Etapa 3: Filtragem por Open Access")
    df_clean = filtrar_registros_open_access(df_clean)
    
    # 4. Salvar e disponibilizar dados processados
    dataset_processado = df_clean
    print("\n✅ Pipeline de processamento concluído com sucesso!")
    print(f"📊 Registros finais: {len(dataset_processado):,}")
    print(f"📋 Colunas: {len(dataset_processado.columns)}")
    print("\n🔗 Variável 'dataset_processado' disponível para próximos notebooks")
else:
    print("⚠️ Não foi possível iniciar o processamento - dados não disponíveis")


🔄 INICIANDO PIPELINE DE PROCESSAMENTO

📋 Etapa 1: Validação dos dados

🔍 VALIDANDO QUALIDADE DOS DADOS WOS
📄 Abstracts válidos: 38,237/38,323 (99.8%)
📰 Títulos válidos: 38,323/38,323 (100.0%)
📅 Anos válidos: 37,834/38,323 (98.7%)
👥 Autores válidos: 38,323/38,323 (100.0%)
🔄 Duplicatas potenciais: 1,014 (2.6%)
📋 Registros completos: 38,237/38,323 (99.8%)

📊 Qualidade geral dos dados: 99.6/100

🔬 ANÁLISE DE RELEVÂNCIA PARA NANOTECNOLOGIA:
  📰 'nano' em títulos: 8440 menções
📰 Títulos válidos: 38,323/38,323 (100.0%)
📅 Anos válidos: 37,834/38,323 (98.7%)
👥 Autores válidos: 38,323/38,323 (100.0%)
🔄 Duplicatas potenciais: 1,014 (2.6%)
📋 Registros completos: 38,237/38,323 (99.8%)

📊 Qualidade geral dos dados: 99.6/100

🔬 ANÁLISE DE RELEVÂNCIA PARA NANOTECNOLOGIA:
  📰 'nano' em títulos: 8440 menções
  📰 'nanoparticle' em títulos: 1985 menções
  📰 'nanotechnology' em títulos: 11 menções
  📰 'nanomaterial' em títulos: 89 menções
  📰 'coating' em títulos: 15126 menções
  📰 'nanoparticle' em títul

In [7]:
# 💾 Salvamento dos dados processados

def salvar_dados_processados(df: pd.DataFrame, validacao: Dict) -> str:
    """Salva os dados processados e metadados"""
    
    print("\n💾 SALVANDO DADOS PROCESSADOS")
    print("=" * 35)
    
    def convert_numpy_types(obj):
        """Convert numpy types to native Python types for JSON serialization"""
        if isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(i) for i in obj]
        return obj
    
    # Criar timestamp para nomeação
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Definir caminhos
    processed_dir = PATHS.get('processed', './processed/')
    Path(processed_dir).mkdir(parents=True, exist_ok=True)
    
    # Salvar DataFrame principal
    arquivo_dados = f"{processed_dir}dataset_wos_processado_{timestamp}.csv"
    df.to_csv(arquivo_dados, index=False, encoding='utf-8-sig')
    print(f"📊 Dataset salvo em: {arquivo_dados}")
    
    # Verificar se o filtro Open Access foi aplicado
    filtro_oa_aplicado = 'filtrado_open_access' in df.columns
    registros_open_access = int(df['filtrado_open_access'].sum()) if filtro_oa_aplicado else 0
    
    # Salvar metadados
    metadados = {
        'timestamp_processamento': timestamp,
        'total_registros': len(df),
        'total_colunas': len(df.columns),
        'validacao': convert_numpy_types(validacao),
        'colunas': list(df.columns),
        'arquivos_origem': df['arquivo_origem'].unique().tolist() if 'arquivo_origem' in df.columns else [],
        'estatisticas_basicas': {
            'registros_alta_qualidade': int(df['qualidade_alta'].sum()) if 'qualidade_alta' in df.columns else 0,
            'registros_open_access': registros_open_access,
            'filtro_open_access_aplicado': filtro_oa_aplicado,
            'anos_publicacao_range': {
                'min': int(df['Publication Year'].min()) if 'Publication Year' in df.columns and df['Publication Year'].notna().any() else None,
                'max': int(df['Publication Year'].max()) if 'Publication Year' in df.columns and df['Publication Year'].notna().any() else None
            } if 'Publication Year' in df.columns else None
        },
        'processamento': {
            'filtro_open_access': filtro_oa_aplicado,
            'limpeza_aplicada': True,
            'data_processamento': datetime.now().isoformat()
        }
    }
    
    arquivo_metadados = f"{processed_dir}metadados_processamento_{timestamp}.json"
    with open(arquivo_metadados, 'w', encoding='utf-8') as f:
        json.dump(metadados, f, indent=2, ensure_ascii=False)
    print(f"📋 Metadados salvos em: {arquivo_metadados}")
    
    # Criar arquivo de referência mais simples para uso futuro
    arquivo_simples = f"{processed_dir}dataset_wos_latest.csv"
    df.to_csv(arquivo_simples, index=False, encoding='utf-8-sig')
    print(f"🔗 Referência criada: {arquivo_simples}")
    
    # Salvar apenas subset para análises teste
    if len(df) > 1000:
        df_sample = df.sample(n=1000, random_state=42)
        arquivo_sample = f"{processed_dir}dataset_wos_sample_1000.csv"
        df_sample.to_csv(arquivo_sample, index=False, encoding='utf-8-sig')
        print(f"🎲 Sample (1000 registros) salvo: {arquivo_sample}")
    
    print(f"\n✅ Salvamento concluído!")
    print(f"📂 Diretório: {processed_dir}")
    print(f"📊 Total de arquivos criados: 3-4 arquivos")
    
    return arquivo_dados

# Salvar se dados foram processados
if 'df_clean' in locals() and df_clean is not None and 'validacao_resultado' in locals():
    arquivo_salvo = salvar_dados_processados(df_clean, validacao_resultado)
    print("💾 Dados salvos com sucesso")
    
    # Disponibilizar dados para próximos notebooks
    dataset_processado = df_clean
    print("\n🔗 Variável 'dataset_processado' disponível para próximos notebooks")
else:
    print("⚠️ Pulando salvamento - dados não processados completamente")


💾 SALVANDO DADOS PROCESSADOS


📊 Dataset salvo em: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_processado_20250610_121255.csv
📋 Metadados salvos em: /home/delon/Modelos/modeloCenanoInk/data/processed/metadados_processamento_20250610_121255.json
🔗 Referência criada: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_latest.csv
🎲 Sample (1000 registros) salvo: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_sample_1000.csv

✅ Salvamento concluído!
📂 Diretório: /home/delon/Modelos/modeloCenanoInk/data/processed/
📊 Total de arquivos criados: 3-4 arquivos
💾 Dados salvos com sucesso

🔗 Variável 'dataset_processado' disponível para próximos notebooks
🔗 Referência criada: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_latest.csv
🎲 Sample (1000 registros) salvo: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_sample_1000.csv

✅ Salvamento concluído!
📂 Diretório: /home/delon/Modelos/modeloCenanoInk/data/processed/
📊 Total de arquivos criados: 3-4 arq

In [8]:
# 📊 Resumo final do carregamento

print("📊 RESUMO FINAL DO CARREGAMENTO E PROCESSAMENTO")
print("=" * 55)

if 'dataset_processado' in locals():
    df_final = dataset_processado
    
    print(f"✅ Status: Processamento concluído com sucesso")
    print(f"📊 Registros finais: {len(df_final):,}")
    print(f"📋 Colunas: {len(df_final.columns)}")
    
    # Destacar informação sobre Open Access
    if 'filtrado_open_access' in df_final.columns:
        oa_count = df_final['filtrado_open_access'].sum()
        if oa_count == len(df_final):
            print(f"🔒 Registros Open Access: {oa_count:,} (100% dos registros)")
        else:
            print(f"🔒 Registros Open Access: {oa_count:,} ({oa_count/len(df_final)*100:.1f}% dos registros)")
    else:
        print(f"🔒 Filtro Open Access: Não aplicado")
    
    if 'qualidade_alta' in df_final.columns:
        alta_qualidade = df_final['qualidade_alta'].sum()
        print(f"⭐ Registros alta qualidade: {alta_qualidade:,} ({alta_qualidade/len(df_final)*100:.1f}%)")
    
    if 'Publication Year' in df_final.columns:
        anos_validos = df_final['Publication Year'].notna().sum()
        if anos_validos > 0:
            ano_min = int(df_final['Publication Year'].min())
            ano_max = int(df_final['Publication Year'].max())
            print(f"📅 Período de publicação: {ano_min} - {ano_max}")
    
    # Verificar relevância para nanotecnologia
    if 'validacao_resultado' in locals():
        nano_titulo = validacao_resultado.get('nano_relevancia_titulo', 0)
        nano_abstract = validacao_resultado.get('nano_relevancia_abstract', 0)
        print(f"🔬 Relevância nano (títulos): {nano_titulo:,} menções")
        print(f"🔬 Relevância nano (abstracts): {nano_abstract:,} menções")
    
    # Pipeline completo
    pipeline_steps = [
        "1. 📥 Carregamento de dados brutos",
        "2. 🗺️ Mapeamento e padronização de colunas", 
        "3. 🔍 Validação de qualidade dos dados",
        "4. 🧹 Limpeza e preprocessamento básico",
        "5. 🔒 Filtragem por Open Access",
        "6. 💾 Salvamento e geração de metadados"
    ]
    print(f"\n🔄 Pipeline completo:")
    for step in pipeline_steps:
        print(f"   ✅ {step}")
    
    print(f"\n🎯 PRÓXIMOS PASSOS:")
    print(f"   1. Execute '03_analise_regex.ipynb' para análise de padrões")
    print(f"   2. Ou execute '04_analise_gemini.ipynb' para análise com IA")
    print(f"   3. Dados disponíveis na variável 'dataset_processado'")
    
    # Estatísticas rápidas de amostra
    print(f"\n📋 Amostra dos dados (primeiros 3 registros):")
    colunas_mostrar = ['Article Title', 'Abstract', 'Publication Year']
    colunas_existentes = [col for col in colunas_mostrar if col in df_final.columns]
    
    if colunas_existentes:
        for i in range(min(3, len(df_final))):
            print(f"\n   📄 Registro {i+1}:")
            for col in colunas_existentes:
                valor = str(df_final.iloc[i][col])[:100]
                print(f"      {col}: {valor}...")
else:
    print("❌ Status: Processamento não foi concluído")
    print("💡 Verifique os passos anteriores para identificar problemas")

print("\n🎉 Notebook de carregamento concluído!")

📊 RESUMO FINAL DO CARREGAMENTO E PROCESSAMENTO
✅ Status: Processamento concluído com sucesso
📊 Registros finais: 9,046
📋 Colunas: 76
🔒 Registros Open Access: 9,046 (100% dos registros)
📅 Período de publicação: 2014 - 2025
🔬 Relevância nano (títulos): 25,973 menções
🔬 Relevância nano (abstracts): 50,949 menções

🔄 Pipeline completo:
   ✅ 1. 📥 Carregamento de dados brutos
   ✅ 2. 🗺️ Mapeamento e padronização de colunas
   ✅ 3. 🔍 Validação de qualidade dos dados
   ✅ 4. 🧹 Limpeza e preprocessamento básico
   ✅ 5. 🔒 Filtragem por Open Access
   ✅ 6. 💾 Salvamento e geração de metadados

🎯 PRÓXIMOS PASSOS:
   1. Execute '03_analise_regex.ipynb' para análise de padrões
   2. Ou execute '04_analise_gemini.ipynb' para análise com IA
   3. Dados disponíveis na variável 'dataset_processado'

📋 Amostra dos dados (primeiros 3 registros):

   📄 Registro 1:
      Article Title: Functionally Gradient Coating of Aluminum Alloy via In Situ Arc Surface Nitriding with Subsequent Fr...
      Abstract: Func